In [1]:
# Import the standard modules
import sqlite3

# Import installed modules
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [3]:
# Connect to the comet database. This database has been created in tutorial
# part 7, however, due to its small size the database is uploaded on GitHub
con = sqlite3.connect('../databases/comets/mpc_comets.db')

# Create a pandas dataframe that contains the perihelion and the absolute
# magnitude
comets_df = pd.read_sql('SELECT PERIHELION_AU, ABSOLUTE_MAGNITUDE ' \
                        'FROM comets_main WHERE ECCENTRICITY < 1', \
                        con)